In [1]:
from os.path import join as pathJoin
import pandas as pd
import numpy as np
import pickle
import scipy.spatial.distance as dist
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
import itertools as itt

In [3]:
data_root = "../tag-genome"

MovieID_TagID_Relevance = pathJoin(data_root, "tag_relevance.dat")
MovieID_Title_MoviePopularity = pathJoin(data_root, "movies.dat")
TagID_Tag_TagPopularity = pathJoin(data_root, "tags.dat")

In [4]:
tag_relevance = pd.read_csv(MovieID_TagID_Relevance, delimiter='\t', header=None, names=['MovieID', 'TagID', 'Relevance'])
movies = pd.read_csv(MovieID_Title_MoviePopularity, delimiter='\t', header=None, names=['MovieID', 'Title', 'MoviePopularity'])
tags = pd.read_csv(TagID_Tag_TagPopularity, delimiter='\t', header=None, names=['TagID', 'Tag', 'TagPopularity'])

In [5]:
with open('../tag-genome/pickled/genome.pickle', 'rb') as f:
    genome = pickle.load(f)

In [60]:
m = movies[movies.Title == "12 Angry Men (1957)"]
# m.Title.iloc[0]
movies.Title


0                                        Toy Story (1995)
1                                          Jumanji (1995)
2                                 Grumpier Old Men (1995)
3                                Waiting to Exhale (1995)
4                      Father of the Bride Part II (1995)
5                                             Heat (1995)
6                                          Sabrina (1995)
7                                     Tom and Huck (1995)
8                                     Sudden Death (1995)
9                                        GoldenEye (1995)
10                         American President, The (1995)
11                     Dracula: Dead and Loving It (1995)
12                                           Balto (1995)
13                                           Nixon (1995)
14                                Cutthroat Island (1995)
15                                          Casino (1995)
16                           Sense and Sensibility (1995)
17            

In [53]:
def popularity(tagId):
    return tags.TagPopularity[tagId] + 1

"Hobbit: The Desolation of Smaug The (2013)".rsplit('(', 1)[0].rsplit(',', 1)[0]

'Hobbit: The Desolation of Smaug The '

In [14]:
def docFreq(tag):
    byTag = genome[:, tag]
    specific = [x for x in byTag if x > 0.5]
    return len(specific) + 2

In [15]:
tag_weights = np.array([np.log(popularity(tagId)) / np.log(docFreq(tagId)) for tagId in range(tags.shape[0])])

In [20]:
# need to get rid of inf
sorted(tag_weights, reverse=True)[:5]

[6.8201789624151887,
 6.1699250014423122,
 4.5235619560570131,
 3.236217269879349,
 2.2435894449529292]

In [13]:
infID = np.argmax(tag_weights)
tag_weights[infID] = 9

In [17]:
def weightedCosSimi(u, v):
    '''Not exactly like in the article'''
    return dist.cosine(u * tag_weights, v * tag_weights)

In [21]:
def articleCosSimi(u, v):
    '''Exactly like in the article'''
    w = tag_weights
    x = np.sum(w * u * v)
    y = np.sqrt(sum(w * u * u)) * np.sqrt(sum(w * v * v)) 
    return  x / y

In [22]:
def rel(tagId, movId):
    return genome[movId, tagId]

In [23]:
# def critiqueDist(critiquedMovieId, retrievedMovieId, tagId, direction):
#     ic, ir, t, d  = critiquedMovieId, retrievedMovieId, tagId, direction
#     return max(0, rel(t, ir) - rel(t, ic) * d)

In [24]:
def printNeigh(randMovNum, indices):
    neighbors = indices[randMovNum]
    for mid in neighbors:
        print(movies.loc[mid]["Title"])

In [37]:
X = genome.copy()
nbrs_art2 = NearestNeighbors(algorithm="brute", metric=weightedCosSimi, n_neighbors=250)
nbrs_art2.fit(X)
distances_art2, indices_art2 = nbrs_art2.kneighbors(X)

NearestNeighbors(algorithm='brute', leaf_size=30,
         metric=<function articleCosSimi at 0x7f774cd98400>,
         metric_params=None, n_jobs=1, n_neighbors=250, p=2, radius=1.0)

In [ ]:
with open('pickled/KNN-neib=250-metr=article.pickle', 'wb') as f:
    pickle.dump(nbrs_art2, f)

In [269]:
movies.head()

,MovieID,Title,MoviePopularity
0,1,Toy Story (1995),53059
1,2,Jumanji (1995),22466
2,3,Grumpier Old Men (1995),15111
3,4,Waiting to Exhale (1995),2898
4,5,Father of the Bride Part II (1995),14323


In [283]:
def N(i):
    return indices_art2[randMovNum]

def Np(i, t):
    delta = 0.23
    return [j for j in N(i) if rel(t, j) > rel(t, i) + delta]

def Nn(i, t):
    delta = 0.23
    return [j for j in N(i) if rel(t, j) < rel(t, i) - delta]

def Nz(i, t):
    return [j for j in N(i) if j not in Np(i, t) and  j not in Nn(i, t)]

In [ ]:
def critiqueEntropy(t, i):
    Nd = np.array(Nn(i, t), Nz(i, t), Np(i, t))
    modNd = [len(Nx) for Nx in Nd]
    modN = len(N(i))
    return np.sum(- modNd / modN * np.log(modNd / modN))

In [286]:
def idToNum(num):
    pass

In [289]:
mId = 0
movies.loc[mId]['Title']

'Toy Story (1995)'

In [292]:
tags.head()

,TagID,Tag,TagPopularity
0,0,007,61
1,1,007 (series),24
2,2,18th century,37
3,3,1920s,42
4,4,1930s,55


In [ ]:
def tagSim(tA, tB):
    relevanceA = genome[:, tA]
    relevanceB = genome[:, tB]
    return dist.cosine(relevanceA, relevanceB)

In [ ]:
# function which we want to minimize by S
def objective_function(S, i):
    "|S| == 5"
    cond1(t) = popularity(t) >= 50
    cond2(t) = np.all(np.array([tagSim(t, u) < 0.5 for u in S if t != u]))
    cond3(t) = critiqueEntropy(t) > 0.325
    ts = [t for t in S if cond1(t) and cond2(t) and cond3(t)]
    iss = list(itt.repeat(i, 5))
    return np.sum(critiqueEntropy(t, i) * np.log(popularity(t)))

In [11]:
def critiqueDist(critiquedMovieId, retrievedMovieId, tagId, direction):
    ic, ir, t, d  = critiquedMovieId, retrievedMovieId, tagId, direction
    return max(0, rel(t, ir) - rel(t, ic) * d)

In [13]:
# for uniform notation
def linearSat(ic, ir, t, d):
    critiqueDist(ic, ir, t,  d)

In [14]:
def diminishSat(ic, ir, t, d):
    1 - np.exp(-5 * critiqueDist(ic, ir, t, d))

In [6]:
import os.path, sys
sys.path.append("/home/esengie/Programming/au/IR/movie_searcher/")
from internet.movies import *

f, g = imdb_descr_and_poster_from_title("Balto")


In [8]:
g

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x05\x03\x04\x04\x04\x03\x05\x04\x04\x04\x05\x05\x05\x06\x07\x0c\x08\x07\x07\x07\x07\x0f\x0b\x0b\t\x0c\x11\x0f\x12\x12\x11\x0f\x11\x11\x13\x16\x1c\x17\x13\x14\x1a\x15\x11\x11\x18!\x18\x1a\x1d\x1d\x1f\x1f\x1f\x13\x17"$"\x1e$\x1c\x1e\x1f\x1e\xff\xdb\x00C\x01\x05\x05\x05\x07\x06\x07\x0e\x08\x08\x0e\x1e\x14\x11\x14\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\x1e\xff\xc0\x00\x11\x08\x01\xb4\x01,\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1d\x00\x00\x01\x05\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x02\x03\x04\x06\x07\x01\x00\x08\t\xff\xc4\x00V\x10\x00\x01\x03\x03\x03\x01\x05\x04\x04\t\x06\x08\r\x05\x01\x01\x01\x02\x03\x04\x00\x05\x11\x06\x12!1\x07\x13AQa\x14"q\x81\x152\x91\xa1\x08#3BR\xb1\xb3\xc1\xd1\x16br\x82\x92\xb2$%4CDSu\xd3\x17&56